### Quick Refresher on Matrix Math, Rotations and Transformations 
 

In [1]:
import numpy as np 
import numpy.typing as npt
from scipy.spatial.transform import Rotation as R


If **A** is an \( m by n \) matrix and **B** is an \( n by p \) matrix,

$$
\mathbf{A} = \begin{pmatrix}
a_{11} & a_{12} & \cdots & a_{1n} \\
a_{21} & a_{22} & \cdots & a_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
a_{m1} & a_{m2} & \cdots & a_{mn}
\end{pmatrix}, \quad
\mathbf{B} = \begin{pmatrix}
b_{11} & b_{12} & \cdots & b_{1p} \\
b_{21} & b_{22} & \cdots & b_{2p} \\
\vdots & \vdots & \ddots & \vdots \\
b_{n1} & b_{n2} & \cdots & b_{np}
\end{pmatrix}
$$

the **matrix product** \( C =AB \)  is defined to be the \( m by p \) matrix

$$
\mathbf{C} = \begin{pmatrix}
c_{11} & c_{12} & \cdots & c_{1p} \\
c_{21} & c_{22} & \cdots & c_{2p} \\
\vdots & \vdots & \ddots & \vdots \\
c_{m1} & c_{m2} & \cdots & c_{mp}
\end{pmatrix}
\quad \text{such that} \quad
c_{ij} = a_{i1}b_{1j} + a_{i2}b_{2j} + \cdots + a_{in}b_{nj}
= \sum_{k=1}^{n} a_{ik} b_{kj}
$$

for \( i = 1, \dots, m \) and \( j = 1, \dots, p \).





Therefore, **AB** can also be written as

$$
\mathbf{C} =
\begin{pmatrix}
a_{11}b_{11} + \cdots + a_{1n}b_{n1} & a_{11}b_{12} + \cdots + a_{1n}b_{n2} & \cdots & a_{11}b_{1p} + \cdots + a_{1n}b_{np} \\
a_{21}b_{11} + \cdots + a_{2n}b_{n1} & a_{21}b_{12} + \cdots + a_{2n}b_{n2} & \cdots & a_{21}b_{1p} + \cdots + a_{2n}b_{np} \\
\vdots & \vdots & \ddots & \vdots \\
a_{m1}b_{11} + \cdots + a_{mn}b_{n1} & a_{m1}b_{12} + \cdots + a_{mn}b_{n2} & \cdots & a_{m1}b_{1p} + \cdots + a_{mn}b_{np}
\end{pmatrix}
$$

Thus the product AB is defined if and only if the number of columns in A equals the number of rows in B,in this case n.

In [2]:
# Matrix Multiplication
def dot_1d(arr1:npt.NDArray, arr2:npt.NDArray): 
    assert arr1.shape == arr2.shape
    assert arr1.ndim == 1 or len(arr1.shape) == 1
    s = 0
    for i in range(len(arr1)): 
        s += arr1[i]* arr2[i]
    return s

def matmul(matrix_a:npt.NDArray, matrix_b:npt.NDArray):
    assert matrix_a.ndim == 2 and matrix_b.ndim == 2
    mat_a_rows, mat_a_cols = matrix_a.shape
    mat_b_rows, mat_b_cols = matrix_b.shape
    assert mat_a_cols == mat_b_rows
    mat_c = np.zeros(shape=(mat_a_rows, mat_b_cols ))
    for i in range(mat_a_rows): 
        for j in range(mat_b_cols): 
            mat_c[i,j] = dot_1d(matrix_a[i], matrix_b[:,j])
    return mat_c

mat_a = np.array([[1,2],[3,4],[3,4]])
mat_b = np.array([[5,6],[7,8]])
assert np.array_equal(mat_a @ mat_b , matmul(mat_a,mat_b))

###  Euler to Rotation Matrix 
Let's say we start out with an Euler matrix of (roll,pitch,yaw), aka rpy. 
This defines a rotation in angular space, but isn't the most friendly to apply in real systems. We can convert this to a rotation matrix that can directly be applied as a mat mul onto a cartesian coordinate system. 

euler angles aren't ideal due to ambiguity in rotation order, and gimbal lock 

Given Euler angles **(roll, pitch, yaw)** corresponding to rotations around the **X**, **Y**, and **Z** axes respectively:

Let:

- \( \phi \): Roll (rotation about X-axis)
- \( \theta \): Pitch (rotation about Y-axis)
- \( \psi \): Yaw (rotation about Z-axis)


**X-axis (Roll):**

R_x =
\begin{bmatrix}
1 & 0 & 0 \\
0 & \cos\phi & -\sin\phi \\
0 & \sin\phi & \cos\phi
\end{bmatrix}


**Y-axis (Pitch):**

R_y =
\begin{bmatrix}
\cos\theta & 0 & \sin\theta \\
0 & 1 & 0 \\
-\sin\theta & 0 & \cos\theta
\end{bmatrix}


**Z-axis (Yaw):**

R_z =
\begin{bmatrix}
\cos\psi & -\sin\psi & 0 \\
\sin\psi & \cos\psi & 0 \\
0 & 0 & 1
\end{bmatrix}

---

### Combined Rotation Matrix (XYZ Order)

R = R_z * R_y * R_x

\[
R = R_z(\psi) \cdot R_y(\theta) \cdot R_x(\phi) =
\begin{bmatrix}
\cos\psi\cos\theta & \cos\psi\sin\theta\sin\phi - \sin\psi\cos\phi & \cos\psi\sin\theta\cos\phi + \sin\psi\sin\phi \\
\sin\psi\cos\theta & \sin\psi\sin\theta\sin\phi + \cos\psi\cos\phi & \sin\psi\sin\theta\cos\phi - \cos\psi\sin\phi \\
-\sin\theta        & \cos\theta\sin\phi                            & \cos\theta\cos\phi
\end{bmatrix}
\]


In [6]:
euler_angle = (10,20,30)
def r_x(roll:float): 
    roll = np.deg2rad(roll)
    return np.array([[1, 0, 0],
            [0, np.cos(roll), -np.sin(roll)],
            [0, np.sin(roll), np.cos(roll)]])
def r_y(pitch:float): 
    pitch = np.deg2rad(pitch)
    return np.array([[np.cos(pitch), 0, np.sin(pitch)],
            [0, 1, 0],
            [-np.sin(pitch),0 ,  np.cos(pitch)]])
def r_z(yaw:float): 
    yaw = np.deg2rad(yaw)
    return np.array([[np.cos(yaw), -np.sin(yaw), 0], 
            [np.sin(yaw), np.cos(yaw), 0], 
            [0,0,1]])

r = R.from_euler('xyz', euler_angle, degrees=True)
rotation_matrix = r.as_matrix()
r_m = r_z(euler_angle[2]) @ r_y(euler_angle[1]) @ r_x(euler_angle[0])
r_mm = matmul(r_z(euler_angle[2]), matmul(r_y(euler_angle[1]), r_x(euler_angle[0])))
print(rotation_matrix)
print(r_m)
print(r_mm)



[[ 0.81379768 -0.44096961  0.37852231]
 [ 0.46984631  0.88256412  0.01802831]
 [-0.34202014  0.16317591  0.92541658]]
[[ 0.81379768 -0.44096961  0.37852231]
 [ 0.46984631  0.88256412  0.01802831]
 [-0.34202014  0.16317591  0.92541658]]
[[ 0.81379768 -0.44096961  0.37852231]
 [ 0.46984631  0.88256412  0.01802831]
 [-0.34202014  0.16317591  0.92541658]]


In [60]:
np.array([1,2,3]) @ rotation_matrix

array([0.72742987, 1.81368636, 3.19082866])